In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model

import datetime

import numpy as np

from plot import vessel_planning

In [2]:
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

In [3]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)


TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
    ),
    {},
)

In [4]:
location_from_site = shapely.geometry.Point(5.1, 52)
location_to_site = shapely.geometry.Point(5, 52.1)
location_to_site_2 = shapely.geometry.Point(5, 52.2)

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "geometry": location_from_site,
    "capacity": 50,
    "level": 50,
}

data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "geometry": location_to_site,
    "capacity": 20,
    "level": 0,
}

data_to_site_2 = {
    "env": my_env,
    "name": "Dumplocatie_2",
    "geometry": location_to_site_2,
    "capacity": 20,
    "level": 0,
}

from_site = Site(**data_from_site)
to_site = Site(**data_to_site)
to_site_2 = Site(**data_to_site_2)

init
level: 50
completed init
init
level: 0
completed init
init
level: 0
completed init


In [5]:
data_hopper = {
    "env": my_env,
    "name": "Hopper 01",
    "geometry": location_from_site,
    "capacity": 5,
    "compute_v": lambda x: 10
}

hopper = TransportProcessingResource(**data_hopper)

init
level: 0
completed init


In [6]:
single_run = [
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing empty",
        "registry": registry,
        "mover": hopper,
        "destination": from_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer MP",
        "registry": registry,
        "processor": hopper,
        "origin": from_site,
        "destination": hopper,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
    }),
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing filled",
        "registry": registry,
        "mover": hopper,
        "destination": to_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer TP",
        "registry": registry,
        "processor": hopper,
        "origin": hopper,
        "destination": to_site,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
    })
]


sequential_activity_data = {
    "env": my_env,
    "name": "Single run process",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
}
activity = model.SequentialActivity(**sequential_activity_data)

while_activity = model.WhileActivity(**{
    "env": my_env,
    "name": "while",
    "registry": registry,
    "sub_process": activity,
    "condition_event": [{"type":"container", "concept": to_site, "state":"full"}],
    "postpone_start": True,
})

while Activity keep_resources []


In [7]:
single_run = [
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing empty 2",
        "registry": registry,
        "mover": hopper,
        "destination": from_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer MP 2",
        "registry": registry,
        "processor": hopper,
        "origin": from_site,
        "destination": hopper,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
    }),
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing filled 2",
        "registry": registry,
        "mover": hopper,
        "destination": to_site_2,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer TP 2",
        "registry": registry,
        "processor": hopper,
        "origin": hopper,
        "destination": to_site_2,
        "amount": 4,
        "duration": 3600,
        "postpone_start": True,
    })
]


activity = model.SequentialActivity(**{
    "env": my_env,
    "name": "Single run process 2",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
})

while_activity_2 = model.WhileActivity(**{
    "env": my_env,
    "name": "while",
    "registry": registry,
    "sub_process": activity,
    "condition_event": [{"type":"container", "concept": to_site_2, "state":"full"}],
    "postpone_start": True,
})

while Activity keep_resources []


In [8]:
sequential_activity_data = {
    "env": my_env,
    "name": "Sequential process",
    "registry": registry,
    "sub_processes": [while_activity, while_activity_2],
}
activity = model.SequentialActivity(**sequential_activity_data)

start event instance None


In [9]:
my_env.run()

keep_resources []
get_full_event : default
start get_available
start event instance None
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
put_callback - id_ default
{}
put_callback - id_ default
{'default': {20: <Event() object at 0x29d4c017ac8>}}
amount :20
put_callback - id_ default
{}
put_callback - id_ default
{}
Mover_move after mover resource request
Mover_move after move
Activity end()
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.6ef7c668-aa2a-11ea-a8bd-2079183a5c9a': 50, 'destination.6efaac1a-aa2a-11ea-ad1a-2079183a5c9a': 5}
destination request : {<simpy.resources.resource.Resource object at 0x0000029D4694DE48>: <Request() object at 0x29d4c017548>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x0000029D4694DE48>: <Request() object at 0x29d4c017548>}
site request : {<s

Mover_move after move
Activity end()
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.6ef7c668-aa2a-11ea-a8bd-2079183a5c9a': 34, 'destination.6efaac1a-aa2a-11ea-ad1a-2079183a5c9a': 5}
destination request : {<simpy.resources.resource.Resource object at 0x0000029D4694DE48>: <Request() object at 0x29d4c039ac8>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x0000029D4694DE48>: <Request() object at 0x29d4c039ac8>}
site request : {<simpy.resources.resource.Resource object at 0x0000029D4694DE48>: <Request() object at 0x29d4c039ac8>, <simpy.resources.resource.Resource object at 0x0000029D4BFB83C8>: <Request() object at 0x29d4bfc2288>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x0000029D4694DE48>: <Request() object at 0x29d4c039ac8>, <simpy.resources.resource.Resource object at 0x0000029D4BFB83C8>: <Request() object at 0x29d4bfc2288>

put_callback - id_ default
{'default': {20: <Event() object at 0x29d4bfb9cc8>}}
amount :20
destination store after put: [{'id': 'default', 'level': 4, 'capacity': 20}]
after put
processor end process
Processed 4 of default:
  by:          Hopper 01
  origin        Hopper 01  contains: 0 of default
  destination:  Dumplocatie_2 contains: 4 of default
after shift amount : {<simpy.resources.resource.Resource object at 0x0000029D4BFBFE88>: <Request() object at 0x29d4bfb5e88>, <simpy.resources.resource.Resource object at 0x0000029D4694DE48>: <Request() object at 0x29d4696f2c8>}
keep resources []
release destination : {<simpy.resources.resource.Resource object at 0x0000029D4694DE48>: <Request() object at 0x29d4696f2c8>}
released origin : {}
released processor : {}
{}
Activity end()
Activity end()
condition event triggered: False False round 0
while loop requested_resources {}
while loop keep_resources []
conditional 
start event instance None
keep_resources []
start event instance None
Mover

In [10]:
pd.DataFrame(hopper.log)

,Message,Timestamp,Value,Geometry,ActivityID,ActivityState
0,sailing empty,1970-01-01 01:00:00.000000,0,POINT (5.1 52),6efc80ca-aa2a-11ea-8fc3-2079183a5c9a,START
1,sailing empty,1970-01-01 01:00:00.000000,0,POINT (5.1 52),6efc80ca-aa2a-11ea-8fc3-2079183a5c9a,STOP
2,transfer default to Hopper 01,1970-01-01 01:00:00.000000,4,POINT (5.1 52),6efc80cb-aa2a-11ea-b147-2079183a5c9a,START
3,transfer default to Hopper 01,1970-01-01 01:00:00.000000,4,POINT (5.1 52),6efc80cb-aa2a-11ea-b147-2079183a5c9a,START
4,transfer default to Hopper 01,1970-01-01 02:00:00.000000,4,POINT (5.1 52),6efc80cb-aa2a-11ea-b147-2079183a5c9a,STOP
...,...,...,...,...,...,...
115,sailing filled,1970-01-02 05:27:08.124207,4,POINT (5 52.2),6efea388-aa2a-11ea-95ac-2079183a5c9a,STOP
116,transfer default to Dumplocatie_2,1970-01-02 05:27:08.124207,4,POINT (5 52.2),6efea389-aa2a-11ea-8bc9-2079183a5c9a,START
117,transfer default to Dumplocatie_2,1970-01-02 05:27:08.124207,4,POINT (5 52.2),6efea389-aa2a-11ea-8bc9-2079183a5c9a,START
118,transfer default to Dumplocatie_2,1970-01-02 06:27:08.124207,4,POINT (5 52.2),6efea389-aa2a-11ea-8bc9-2079183a5c9a,STOP


In [12]:
objects = [hopper]
activities = []
for obj in objects:
    print(set(obj.log["Message"]))
    activities.extend(set(obj.log["Message"]))
            
C = np.linspace(0,255, len(activities))
colors = {}

for i in range(len(activities)):
    colors[i] = f'rgb({int(C[i]/2)},{int(C[len(C) - 1 - i]/1.5)},{int(C[i])})'

vessel_planning(objects, activities, colors)

{'transfer default to Hopper 01', 'transfer default to Dumplocatie', 'sailing empty', 'transfer default to Dumplocatie_2', 'sailing filled'}
